# Exploring Titan with Spectral Analysis

(description)


## 1. Import Libraries

**Numpy**: Managing multi-dimensional arrays and curve fitting. [Numpy Documentation](https://numpy.org/doc/)  
**Matplotlib**: Creating and displaying plots. [Matplotlib Documentation](https://matplotlib.org/3.3.3/contents.html)  
**Pysis**: Reading .cub files into array format. [Pysis Documentation](https://pysis.readthedocs.io/en/latest/)     
**os**: Navigating and viewing directories. [os Documentation](https://docs.python.org/3/library/os.html)  
**SciPy** Fitting Gaussian functions and power laws to data. [SciPy Documentation](https://www.scipy.org/docs.html)


In [1]:
from IPython.core.display import HTML
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from pylab import *
import pysis
import os
import scipy.optimize as sco
import time
import ast
from tkinter import *
%matplotlib qt

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


In [2]:
#Navigation information
'''
0: Number of Counts
1: Phase *
2: Emission * 
3: Incidence  *
4: Local Emission  
5: Local Incidence  
6: Latitude
7: Longitude  
8: Pixel Resolution  
9: Line Resolution  
10: Sample Resolution  
11: Detector Resolution  
12: North Azimuth
13: Sun Azimuth  
14: Spacecraft Azmituh  
15: Off Nadir Angle
16: Subspacecraft Ground Azimuth  
17: Subsolar Ground Azimuth
18: Morphology = Pixel Resolution/cos(Emission Angle)
19: Albedo = PixelResolution * [(1 / cos(EmissionAngle)) + (1 / cos(IncidenceAngle))]
20: Right Ascension
21: Declination
22: Body Fixed Coordinate X
23: Body Fixed Coordinate Y
24: Body Fixed Coordinate Z
'''

vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 

In [3]:
def constant_value_nav(nav, nav_number,value):
    """
    Given navigation data, an index into nav measurements, and a value, returns a set of all (i,j) with that value in nav. 
    """
    constants = []
    for i in range(64):
        for j in range(64):
           # print(i)
            if round(nav[nav_number,i,j]) == value:
                #print(i)
                constants.append((i,j))
            else:
                continue
                
    return constants

In [5]:
def gaussian(x,mean,sigma,A):
    
    return A*np.exp( -((x-mean)**2.) / (2.*sigma**2.) )

def powerlaw(x,a, k, c):
    
    return a*x**k + c



In [6]:
def create_gaussian_array(cube,nav,x,y):
    """
    Given cube data (cube) and navigation information (nav), adjusts data by subtracting fitted powerlaw and returns x by y arrays of:
    
    1) Gaussian means
    2) Gaussian widths
    3) Gaussian amplitudes
    
    4) Powerlaw amplitudes
    5) Powerlaw exponent
    6) Powerlaw constant
    
    """
    
    band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 
    
    max_mean = 0
    max_sigma=0
    max_amp = 0
    means = np.zeros((x,y))
    sigmas = np.zeros((x,y))
    amps = np.zeros((x,y))
    plaw = np.zeros((x,y))
    plaw2 = np.zeros((x,y))
    plaw3 = np.zeros((x,y))
    
    for i in range(x):
        for j in range(y):
            #incidence and emission < 70
            if nav[6,i,j] < -90 and nav[2,i,j]<70 and nav[3,i,j]<70:
                continue
            else:
                try:
                    #Fit power law
                    p_fit, p_cov = sco.curve_fit(powerlaw, vims_wave[band_channels], cube[band_channels, i,j], p0 = (.005,-2,0), maxfev = 10000)
                    p = powerlaw(vims_wave, *p_fit)
                    
                    #Subtract power law
                    new_data = cube[:,i,j] - p

                    #Fit gaussian
                    fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],new_data[64:85], \
                                        p0=(2.08,.125,.1), maxfev=100000)

                    plaw[i,j] = p_fit[0]
                    plaw2[i,j] = p_fit[1]
                    plaw3[i,j] = p_fit[2]
                    
                except:
                    #If powerlaw fit fails, directly fit gaussian
                    fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],cube[64:85,i,j], p0=(2.08,.125,.1), maxfev=10000)
                    
                    
                means[i,j] = fit[0]
                if fit[0]>max_mean:
                    max_mean = fit[0]
                sigmas[i,j] = fit[1] 
                if fit[1]>max_sigma:
                    max_sigma = fit[1]
                amps[i,j] = fit[2]
                if fit[2] > max_amp:
                    max_amp = fit[2]
    
    return (means,sigmas,amps,plaw,plaw2,plaw3,max_mean,max_sigma,max_amp)

In [13]:
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514281143_1_ir_nav.cub').data

plt.imshow(nav[6])

In [7]:
def create_panels(data, nav,x,y,cb=[(0,.2),(0,.2),(2.02,2.04),(27,30),(0,80),(0,100),(0,0.1),(-3,-2)]):  
    """
    Given cube data and navigation information, creates 9 panels summarizing variation of different variables. Cb is a list of tuples representing 
    min and max values of the colorbar for each variable.
    
    1) Gaussian Amplitudes
    2) Gaussian Width
    3) Gaussian Mean
    
    4) Navigation Phase
    5) Navigation Emission
    6) Navigation Incidence
    
    7) Powerlaw Amplitude
    8) Powerlaw Exponent
    
    9) Channel 69 
    """
    t0=time.time()
    array = create_gaussian_array(data,nav,x,y)
    #print(array[6])
    #print(array[7])
    #print(array[8])
    print(array[2])
    print(np.min(array[2]))
#     print(np.max(array[1]))
#     print(np.max(array[0]))
#     print(np.max(array[2]))
    print(f'Time: {time.time()-t0}')
    A = array[2]
    W = array[1]
    M = array[0]
    plaw_amp = array[3]
    plaw_power = array[4]
    plaw_constant = array[5]
    #Subplots
    f = plt.figure(figsize=(10,3))
    ax = f.add_subplot(331)
    ax2 = f.add_subplot(332)
    ax3 = f.add_subplot(333)
    ax4 = f.add_subplot(334)
    ax5 = f.add_subplot(335)
    ax6 = f.add_subplot(336)
    ax7 = f.add_subplot(337)
    ax8 = f.add_subplot(338)
    ax9 = f.add_subplot(339)

    #GAUSSIAN
    ax.title.set_text("Gaussian - Amplitudes")
    amps = ax.imshow(A,vmin=cb[0][0],vmax=cb[0][1])
    plt.colorbar(amps,ax=ax)

    ax2.title.set_text("Gaussian - Width")
    width = ax2.imshow(W, vmin = cb[1][0], vmax=cb[1][1])
    plt.colorbar(width,ax=ax2)

    ax3.title.set_text("Gaussian - Mean")
    mean = ax3.imshow(M, vmin=cb[2][0],vmax=cb[2][1])
    plt.colorbar(mean,ax=ax3)

    #NAVIGATION
    ax4.title.set_text("Navigation - Phase")
    phase = ax4.imshow(nav[1,:,:], vmin=cb[3][0],vmax=cb[3][1])
    plt.colorbar(phase, ax=ax4)

    ax5.title.set_text("Navigation - Emission")
    emission = ax5.imshow(nav[2,:,:], vmin=cb[4][0], vmax=cb[4][1])
    plt.colorbar(emission,ax = ax5)

    ax6.title.set_text("Navigation - Incidence")
    incidence = ax6.imshow(nav[3,:,:], vmin = cb[5][0], vmax=cb[5][1])
    plt.colorbar(incidence,ax=ax6)

    ax7.title.set_text("Powerlaw - Amplitude")
    plaw = ax7.imshow(plaw_amp, vmin = cb[6][0], vmax=cb[6][1])
    plt.colorbar(plaw,ax=ax7)

    ax8.title.set_text("Powerlaw - Exponent")
    plaw2 = ax8.imshow(plaw_power, vmin=cb[7][0],vmax=cb[7][1])
    plt.colorbar(plaw2,ax=ax8)

    ax9.title.set_text("Channel 69")
    channel = ax9.imshow(data[69,:,:])
    plt.colorbar(channel,ax=ax9)
    #return array

In [8]:
%matplotlib qt
colorbar = [(0,.2),(0,.2),(2.02,2.04),(27,30),(0,80),(0,100),(0,0.1),(-3,-2)]

data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280412_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280412_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280412_1_ir_nav.cub').data

create_panels(data,nav,64,64,colorbar)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03005447 0.021433   0.        ]
 [0.         0.         0.         ... 0.02654463 0.01766282 0.        ]
 [0.         0.         0.         ... 0.0218084  0.01362918 0.        ]]
0.0
Time: 2.542465925216675


In [9]:
colorbar = [(0,.2),(0,.2),(2.02,2.04),(27,30),(0,80),(0,100),(0,0.1),(-3,-2)]

data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280779_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280779_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280779_1_ir_nav.cub').data

create_panels(data,nav,64,64,colorbar)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0
Time: 0.0315251350402832


In [22]:
data8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2016096T054156_2016097T040514/CM_1838527237_1_ir.cub').data
lbl8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2016096T054156_2016097T040514/CM_1838527237_1_ir.cub').label
nav8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2016096T054156_2016097T040514/CM_1838527237_1_ir_nav.cub').data

create_panels(data8,nav8,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
-4.86008336545381e+55
Time: 15.74007534980774


In [15]:
#Viewing geometry - 2009189T194600_2009189T235739
colorbar_2009 = [(0,0.05),(0.07,0.09),(2.02,2.04),(28.1,28.3),(50,90),(85,110),(0.01,0.03),(-1.5,-3)]


data8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739/CM_1625776676_1_ir.cub').data
lbl8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739/CM_1625776676_1_ir.cub').label
nav8 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739/CM_1625776676_1_ir_nav.cub').data


create_panels(data8,nav8,lbl8['IsisCube']['Instrument']['SwathLength'],lbl8['IsisCube']['Instrument']['SwathWidth'])#, colorbar_2009)

[[0.05549759 0.05453765 0.05458778 0.05281346 0.0482555  0.04738456]
 [0.05645824 0.05652878 0.05525718 0.06289789 0.04913664 0.05052656]
 [0.05685438 0.05704925 0.05712555 0.05339248 0.05061198 0.04924662]
 [0.05755528 0.05873596 0.0573567  0.05470835 0.05408849 0.05059866]
 [0.05838394 0.05883187 0.05778497 0.05722954 0.05455792 0.05096411]
 [0.05921244 0.05944073 0.05756855 0.05589116 0.05417915 0.05387584]
 [0.05756188 0.0567975  0.0547277  0.05233237 0.05222011 0.0499298 ]
 [0.05617702 0.05293171 0.05164898 0.04837762 0.04604603 0.04219691]
 [0.05661732 0.05439255 0.04826203 0.0470628  0.04776883 0.04416011]
 [0.06194283 0.06539657 0.06071552 0.06191081 0.05966165 0.04988151]
 [0.06203798 0.06977007 0.06161995 0.06546066 0.06236797 0.05323149]
 [0.05891612 0.05967255 0.06106165 0.06114871 0.05898153 0.051708  ]
 [0.05550852 0.05837196 0.06334007 0.0593348  0.05402275 0.04898217]
 [0.056631   0.0605126  0.06218362 0.05794154 0.05466255 0.04903763]
 [0.05756528 0.06016771 0.06199255

In [18]:
#Different viewing geometry - 2016127T223449_2016130T200342
%matplotlib qt

colorbar_2016 = [(0,.3),(-0.001,0.2),(2.02,2.05),(120,122),(0,80),(0,130),(-0.05,0.15),(-3,0)]


data7 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').data
lbl7 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').label
nav7 = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841286170_1_ir_nav.cub').data

create_panels(data7, nav7,46,32,colorbar_2016)

[[0.         0.         0.         ... 0.00060426 0.00126782 0.00157523]
 [0.         0.         0.         ... 0.00108926 0.00208359 0.00171547]
 [0.         0.         0.         ... 0.00063724 0.00421932 0.0014823 ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
-1.6982061423742842e+49
Time: 7.340968370437622


In [7]:
def create_2D_plots(folder_directories, axes, xlims=None,ylims=None):
    """
    Given a directory containing cub files, creates a plot between y (means, sigmas, amps) and x (navigation variable).
    
    Gaussian Key:
    'Mean':0,
    'Sigma': 1,
    'Amplitudes':2
    
    Navigation Key:
    'Number of Counts':0,
    'Phase':1,
    'Emission':2, 
    'Incidence':3, 
    'Local Emission':4 , 
    'Local Incidence':5,
    'Latitude': 6,
    'Longitude': 7,  
    'Pixel Resolution': 8,  
    'Line Resolution': 9,  
    'Sample Resolution': 10,  
    'Detector Resolution': 11,  
    'North Azimuth': 12,
    'Sun Azimuth': 13,  
    'Spacecraft Azmituh': 14, 
    'Off Nadir Angle': 15,
    'Subspacecraft Ground Azimuth': 16,
    'Subsolar Ground Azimuth': 17,
    'Morphology': 18,
    'Albedo': 19,
    'Right Ascension': 20,
    'Declination': 21,
    'Body Fixed Coordinate X': 22,
    'Body Fixed Coordinate Y': 23,
    'Body Fixed Coordinate Z': 24
    
    
    """
    ref = {'Mean':0, 'Sigma': 1, 'Amplitudes':2}

    nav_info = {
        'Number of Counts': 0,
        'Phase': 1,
        'Emission': 2,
        'Incidence': 3,
        'Local Emission': 4,
        'Local Incidence': 5,
        'Latitude': 6,
        'Longitude': 7,
        'Pixel Resolution': 8,
        'Line Resolution': 9,
        'Sample Resolution': 10,
        'Detector Resolution': 11,
        'North Azimuth': 12,
        'Sun Azimuth': 13,
        'Spacecraft Azmituh': 14,
        'Off Nadir Angle': 15,
        'Subspacecraft Ground Azimuth': 16,
        'Subsolar Ground Azimuth': 17,
        'Morphology': 18,
        'Albedo': 19,
        'Right Ascension': 20,
        'Declination': 21,
        'Body Fixed Coordinate X': 22,
        'Body Fixed Coordinate Y': 23,
        'Body Fixed Coordinate Z': 24
    }
    #get unique
    params = {}
    for x,y in axes:
        assert (y in ref or y in nav_info) and (x in ref or x in nav_info), 'Invalid axes'
        if x not in params:
            params[x]={'is_nav': x in nav_info, 'vals':[]}
        if y not in params:
            params[y]={'is_nav': y in nav_info, 'vals':[]}
        
    fig=plt.figure()
    num_of_subplots=len(axes)
    rows=num_of_subplots//3 if num_of_subplots%3==0 else num_of_subplots//3+1
    ref = {'Mean': 0, 'Sigma': 1, 'Amplitudes': 2}
    

    failed = 0
    done = 0

        

    #loop through files
    for folder_directory in folder_directories:
        print(f"In directory: {folder_directory}")
        files = os.listdir(folder_directory)
        for file in files:
            if file[0:2] != 'CM' or 'nav' in file or 'BKGD' in file:
                continue
            else:
                print(f'Working on: {file}')
                try:
                    #create guassian array for each file

                    data = pysis.CubeFile(f'{folder_directory}/{file}').data
                    lbl = pysis.CubeFile(f'{folder_directory}/{file}').label
                    nav = pysis.CubeFile(
                        f'{folder_directory}/{file[:-4]}_nav.cub').data
                    length = lbl['IsisCube']['Instrument']['SwathLength']
                    width = lbl['IsisCube']['Instrument']['SwathWidth']
                    vals = create_gaussian_array(
                        data, nav, lbl['IsisCube']['Instrument']['SwathLength'],
                        lbl['IsisCube']['Instrument']['SwathWidth'])

                    for i in range(length):
                        for j in range(width):
                            if nav[6, i, j] < -90:
                                continue
                            for param in params:
                                if params[param]['is_nav']:
                                    params[param]['vals'].append(nav[nav_info[param],i,j])
                                    
                                else:
                                    params[param]['vals'].append(vals[ref[param]][i,j])

                    done += 1

                except Exception as e:
                    failed += 1
                    print(
                        f'File {file} out of {failed} failures and {done} completed with error {e}'
                    )

        #Print summary and display scatter plot
        print('\n====SUMMARY====')
        print(f'Files failed: {failed}')
        print(f'Files done: {done}')
        print(f'Total files: {failed+done}')
        print('===============\n')
                
    for i in range(len(axes)):
        index=i+1
        ax = fig.add_subplot(rows,3,index)


        ax.scatter(params[axes[i][0]]['vals'], params[axes[i][1]]['vals'], s=2)
        ax.set_xlabel(axes[i][0])
        ax.set_ylabel(axes[i][1])
        if xlims != None and index in xlims:
            ax.set_xlim(xlims[index][0], xlims[index][1])
        if ylims != None and index in ylims:
            ax.set_ylim(ylims[index][0], ylims[index][1])
        ax.title.set_text(f'{axes[i][0]} and {axes[i][1]}')
        
    plt.subplots_adjust(wspace=0.5,hspace=0.5)
    
    plt.show()

In [8]:
def create_3D_plots(folder_directories, axes, xlims=None,ylims=None,zlims=None):
    """
    Given a directory containing cub files, creates a plot between y (means, sigmas, amps) and x (navigation variable).
    
    Gaussian Key:
    'Mean':0,
    'Sigma': 1,
    'Amplitudes':2
    
    Navigation Key:
    'Number of Counts':0,
    'Phase':1,
    'Emission':2, 
    'Incidence':3, 
    'Local Emission':4 , 
    'Local Incidence':5,
    'Latitude': 6,
    'Longitude': 7,  
    'Pixel Resolution': 8,  
    'Line Resolution': 9,  
    'Sample Resolution': 10,  
    'Detector Resolution': 11,  
    'North Azimuth': 12,
    'Sun Azimuth': 13,  
    'Spacecraft Azmituh': 14, 
    'Off Nadir Angle': 15,
    'Subspacecraft Ground Azimuth': 16,
    'Subsolar Ground Azimuth': 17,
    'Morphology': 18,
    'Albedo': 19,
    'Right Ascension': 20,
    'Declination': 21,
    'Body Fixed Coordinate X': 22,
    'Body Fixed Coordinate Y': 23,
    'Body Fixed Coordinate Z': 24
    
    
    """
    ref = {'Mean':0, 'Sigma': 1, 'Amplitudes':2}

    nav_info = {
        'Number of Counts': 0,
        'Phase': 1,
        'Emission': 2,
        'Incidence': 3,
        'Local Emission': 4,
        'Local Incidence': 5,
        'Latitude': 6,
        'Longitude': 7,
        'Pixel Resolution': 8,
        'Line Resolution': 9,
        'Sample Resolution': 10,
        'Detector Resolution': 11,
        'North Azimuth': 12,
        'Sun Azimuth': 13,
        'Spacecraft Azmituh': 14,
        'Off Nadir Angle': 15,
        'Subspacecraft Ground Azimuth': 16,
        'Subsolar Ground Azimuth': 17,
        'Morphology': 18,
        'Albedo': 19,
        'Right Ascension': 20,
        'Declination': 21,
        'Body Fixed Coordinate X': 22,
        'Body Fixed Coordinate Y': 23,
        'Body Fixed Coordinate Z': 24
    }
    #get unique
    params = {}
    for x,y,z in axes:
        assert (z in ref or z in nav_info) and (y in ref or y in nav_info) and (x in ref or x in nav_info), 'Invalid axes'
        if x not in params:
            params[x]={'is_nav': x in nav_info, 'vals':[]}
        if y not in params:
            params[y]={'is_nav': y in nav_info, 'vals':[]}
        if z not in params:
            params[z] = {'is_nav': z in nav_info, 'vals':[]}
    fig=plt.figure()
    num_of_subplots=len(axes)
    rows=num_of_subplots//2 if num_of_subplots%2==0 else num_of_subplots//2+1
    ref = {'Mean': 0, 'Sigma': 1, 'Amplitudes': 2}
    

    failed = 0
    done = 0

        

    #loop through files
    for folder_directory in folder_directories:
        print(f"In directory: {folder_directory}")
        files = os.listdir(folder_directory)
        for file in files:
            if file[0:2] != 'CM' or 'nav' in file or 'BKGD' in file:
                continue
            else:
                print(f'Working on: {file}')
                try:
                    #create guassian array for each file

                    data = pysis.CubeFile(f'{folder_directory}/{file}').data
                    lbl = pysis.CubeFile(f'{folder_directory}/{file}').label
                    nav = pysis.CubeFile(
                        f'{folder_directory}/{file[:-4]}_nav.cub').data
                    length = lbl['IsisCube']['Instrument']['SwathLength']
                    width = lbl['IsisCube']['Instrument']['SwathWidth']
                    vals = create_gaussian_array(
                        data, nav, lbl['IsisCube']['Instrument']['SwathLength'],
                        lbl['IsisCube']['Instrument']['SwathWidth'])

                    for i in range(length):
                        for j in range(width):
                            if nav[6, i, j] < -90:
                                continue
                            for param in params:
                                if params[param]['is_nav']:
                                    params[param]['vals'].append(nav[nav_info[param],i,j])
                                    
                                else:
                                    params[param]['vals'].append(vals[ref[param]][i,j])

                    done += 1

                except Exception as e:
                    failed += 1
                    print(
                        f'File {file} out of {failed} failures and {done} completed with error {e}'
                    )

        #Print summary and display scatter plot
        print('\n====SUMMARY====')
        print(f'Files failed: {failed}')
        print(f'Files done: {done}')
        print(f'Total files: {failed+done}')
        print('===============\n')
                
    for i in range(len(axes)):
        index=i+1
        ax = fig.add_subplot(rows,2,index, projection='3d')


        ax.scatter(params[axes[i][0]]['vals'], params[axes[i][1]]['vals'], params[axes[i][2]]['vals'], s=2)
        ax.set_xlabel(axes[i][0])
        ax.set_ylabel(axes[i][1])
        ax.set_zlabel(axes[i][2])
        if xlims != None and index in xlims:
            ax.set_xlim3d(xlims[index][0], xlims[index][1])
        if ylims != None and index in ylims:
            ax.set_ylim3d(ylims[index][0], ylims[index][1])
        if zlims != None and index in zlims:
            ax.set_zlim3d(zlims[index][0],zlim[index][1])
        ax.title.set_text(f'{axes[i][0]}, {axes[i][1]}, {axes[i][2]}')
        
    plt.subplots_adjust(wspace=0.5,hspace=0.5)

    plt.show()

All plots for 2005

<img src="all_2005_adjusted.png" width="750"/>

2005 3D
<img src="2005_3d.png" width="750"/>

2005, 2016, 2009 2D
<img src='2005&2016&2009_all.png' width='750'/>

In [9]:
def create_all_2D(folders,xlims=None,ylims=None):
    graphs = [('Incidence', 'Amplitudes'),('Incidence','Mean'),('Incidence','Sigma'),('Phase','Amplitudes'),('Phase','Mean'),('Phase','Sigma'),('Emission','Amplitudes'),('Emission','Mean'),('Emission','Sigma')]
    return create_2D_plots(folders,graphs,xlims,ylims)

<img src="amps_incidence.png" width="750"/>


In [13]:
graphs = [('Incidence', 'Amplitudes'),('Incidence','Mean'),('Incidence','Sigma'),('Phase','Amplitudes'),('Phase','Mean'),('Phase','Sigma'),('Emission','Amplitudes'),('Emission','Mean'),('Emission','Sigma')]
ranges = {1:(0,0.4),4:(0,0.4),7:(0,0.4),2:(1.9,2.1),3:(-10,25),6:(-10,25),9:(-10,25),5:(1.5,2.5),8:(1.5,2.5)}
create_2D_plots(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919','/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342','/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739'],graphs,ylims=ranges)    

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub
Working on: CM_1514278619_1_ir.cub
Working on: CM_1514280412_1_ir.cub
Working on: CM_1514284191_1_ir.cub
Working on: CM_1514277541_1_ir.cub
Working on: CM_1514281143_1_ir.cub
Working on: CM_1514282941_1_ir.cub
Working on: CM_1514277901_1_ir.cub

====SUMMARY====
Files failed: 0
Files done: 11
Total files: 11

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342
Working on: CM_1841292710_1_ir.cub


/home/alanyu/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Working on: CM_1841351567_1_ir.cub
Working on: CM_1841281547_1_ir.cub
Working on: CM_1841283690_1_ir.cub
Working on: CM_1841376895_1_ir.cub
Working on: CM_1841278414_1_ir.cub
Working on: CM_1841282090_1_ir.cub
Working on: CM_1841387657_8_ir.cub
Working on: CM_1841281273_1_ir.cub
Working on: CM_1841368069_1_ir.cub
Working on: CM_1841279980_1_ir.cub
Working on: CM_1841284507_1_ir.cub
Working on: CM_1841275013_1_ir.cub
Working on: CM_1841351414_1_ir.cub
Working on: CM_1841283964_1_ir.cub
Working on: CM_1841272581_1_ir.cub
Working on: CM_1841278020_1_ir.cub
Working on: CM_1841269234_1_ir.cub
Working on: CM_1841346809_1_ir.cub
Working on: CM_1841290630_1_ir.cub
Working on: CM_1841387557_1_ir.cub
Working on: CM_1841288944_9_ir.cub
Working on: CM_1841367934_1_ir.cub
Working on: CM_1841286170_1_ir.cub
Working on: CM_1841346547_1_ir.cub
Working on: CM_1841355994_1_ir.cub
Working on: CM_1841268447_1_ir.cub
Working on: CM_1841286987_1_ir.cub
Working on: CM_1841286444_1_ir.cub
Working on: CM_18412

In [10]:
create_all_2D(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919'])

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub


KeyboardInterrupt: 

In [23]:
ranges = {1:(0,0.4),4:(0,0.4),7:(0,0.4),2:(1.9,2.1),3:(-10,25),6:(-10,25),9:(-10,25),5:(1.5,2.5),8:(1.5,2.5)}
create_all_2D(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919','/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342','/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739'],ylims=ranges)

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub
Working on: CM_1514278619_1_ir.cub
Working on: CM_1514280412_1_ir.cub
Working on: CM_1514284191_1_ir.cub
Working on: CM_1514277541_1_ir.cub
Working on: CM_1514281143_1_ir.cub
Working on: CM_1514282941_1_ir.cub
Working on: CM_1514277901_1_ir.cub

====SUMMARY====
Files failed: 0
Files done: 11
Total files: 11

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342
Working on: CM_1841292710_1_ir.cub
Working on: CM_1841351567_1_ir.cub
Working on: CM_1841281547_1_ir.cub
Working on: CM_1841283690_1_ir.cub
Working on: CM_1841376895_1_ir.cub
Working on: CM_1841278414_1_ir.cub
Working on: CM_1841282090_1_ir.cub
Working on: CM_1841387657_8_ir.cub
Working on: CM_1841281273_1_ir.cub
Working on: CM_1841368069_1_ir.cub
Working on: CM_1841279980_1_ir.cub
Wor

In [16]:
create_2D_plots(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919'],[('Incidence','Amplitudes')],ylims={1:(0,0.4)})

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub
Working on: CM_1514278619_1_ir.cub
Working on: CM_1514280412_1_ir.cub
Working on: CM_1514284191_1_ir.cub
Working on: CM_1514277541_1_ir.cub
Working on: CM_1514281143_1_ir.cub
Working on: CM_1514282941_1_ir.cub
Working on: CM_1514277901_1_ir.cub

====SUMMARY====
Files failed: 0
Files done: 11
Total files: 11



In [24]:
create_3D_plots(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919'],[('Amplitudes','Incidence','Emission'),('Mean','Sigma','Incidence')], xlims={1:(0,0.5)})

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub
Working on: CM_1514278619_1_ir.cub
Working on: CM_1514280412_1_ir.cub
Working on: CM_1514284191_1_ir.cub
Working on: CM_1514277541_1_ir.cub
Working on: CM_1514281143_1_ir.cub
Working on: CM_1514282941_1_ir.cub
Working on: CM_1514277901_1_ir.cub

====SUMMARY====
Files failed: 0
Files done: 11
Total files: 11



In [19]:
%matplotlib qt
ranges = {2: (2.02, 2.05), 5: (2.02, 2.05), 8: (2.02, 2.05)}
create_all_2D(
    ['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919'],
    ylims=ranges)

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919
Working on: CM_1514277149_1_ir.cub
Working on: CM_1514280779_1_ir.cub
Working on: CM_1514282576_1_ir.cub
Working on: CM_1514280052_1_ir.cub
Working on: CM_1514278619_1_ir.cub
Working on: CM_1514280412_1_ir.cub
Working on: CM_1514284191_1_ir.cub
Working on: CM_1514277541_1_ir.cub
Working on: CM_1514281143_1_ir.cub
Working on: CM_1514282941_1_ir.cub
Working on: CM_1514277901_1_ir.cub

====SUMMARY====
Files failed: 0
Files done: 11
Total files: 11



In [21]:
ranges = {1:(0,0.4),4:(0,0.4),7:(0,0.4),2:(1.9,2.1),3:(-10,25),6:(-10,25),9:(-10,25)}
create_all_2D(['/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342'],ylims=ranges) #ylim = (0,0.2)) #ylim=(0,0.005))

In directory: /home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342
Working on: CM_1841292710_1_ir.cub
Working on: CM_1841351567_1_ir.cub
Working on: CM_1841281547_1_ir.cub
Working on: CM_1841283690_1_ir.cub
Working on: CM_1841376895_1_ir.cub
Working on: CM_1841278414_1_ir.cub
Working on: CM_1841282090_1_ir.cub
Working on: CM_1841387657_8_ir.cub
Working on: CM_1841281273_1_ir.cub
Working on: CM_1841368069_1_ir.cub
Working on: CM_1841279980_1_ir.cub
Working on: CM_1841284507_1_ir.cub
Working on: CM_1841275013_1_ir.cub
Working on: CM_1841351414_1_ir.cub
Working on: CM_1841283964_1_ir.cub
Working on: CM_1841272581_1_ir.cub
Working on: CM_1841278020_1_ir.cub
Working on: CM_1841269234_1_ir.cub
Working on: CM_1841346809_1_ir.cub
Working on: CM_1841290630_1_ir.cub
Working on: CM_1841387557_1_ir.cub
Working on: CM_1841288944_9_ir.cub
Working on: CM_1841367934_1_ir.cub
Working on: CM_1841286170_1_ir.cub
Working on: CM_1841346547_1_ir.cub
Working on: CM_1841355994_1_ir.cub
Working o

In [20]:
ranges = {1:(0,0.15),4:(0,0.15),7:(0,0.15),2:(1.97,2.1),5:(1.97,2.1),8:(1.97,2.1),3:(0,0.15),6:(0,0.15),9:(0,0.15)}
create_all_plots('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2009189T194600_2009189T235739',ranges) #ylim = (0,0.2)) #ylim=(0,0.005))

NameError: name 'create_all_plots' is not defined

In [26]:
#color as third dimension
#find common region - bounded, uniform. pick from iss map (30 lat/long)
#parse through data and return interesting spectra
#text file - min/max/incidence/emission/lat/lon
#cut - resolution - 100km

nav

array([[[-1.3116009e-04, -2.6019919e-04, -2.5663720e-04, ...,
          1.1544797e-06, -9.5173251e-05, -9.5477466e-05],
        [-4.0532224e-04,  1.2109559e-04, -2.6570930e-04, ...,
         -1.9787112e-04, -1.9842901e-04, -6.7042142e-06],
        [-6.7663286e-04, -1.9608693e-05, -4.0241185e-04, ...,
         -2.0476787e-04, -2.0533154e-04, -2.0595448e-04],
        ...,
        [-1.1923433e-04, -2.3856988e-04, -1.1620991e-04, ...,
          3.8877673e-02,  3.2530483e-02,  2.6245134e-02],
        [-2.5234898e-04, -2.4910067e-04, -2.4600117e-04, ...,
          3.6503118e-02,  3.0411521e-02,  2.3524377e-02],
        [-1.3997166e-04, -1.3818547e-04, -4.9658370e-04, ...,
          3.4403015e-02,  2.7708951e-02,  2.0599358e-02]],

       [[-3.4028227e+38, -3.4028227e+38, -3.4028227e+38, ...,
         -3.4028227e+38, -3.4028227e+38, -3.4028227e+38],
        [-3.4028227e+38, -3.4028227e+38, -3.4028227e+38, ...,
         -3.4028227e+38, -3.4028227e+38, -3.4028227e+38],
        [-3.4028227e+38, 

## Cube Parser

0: Number of Counts  
1: Phase *  
2: Emission *   
3: Incidence  *  
4: Local Emission  
5: Local Incidence  
6: Latitude
7: Longitude  
8: Pixel Resolution  
9: Line Resolution  
10: Sample Resolution  
11: Detector Resolution  
12: North Azimuth
13: Sun Azimuth  
14: Spacecraft Azmituh  
15: Off Nadir Angle
16: Subspacecraft Ground Azimuth  
17: Subsolar Ground Azimuth
18: Morphology = Pixel Resolution/cos(Emission Angle)
19: Albedo = PixelResolution * [(1 / cos(EmissionAngle)) + (1 / cos(IncidenceAngle))]
20: Right Ascension
21: Declination
22: Body Fixed Coordinate X
23: Body Fixed Coordinate Y
24: Body Fixed Coordinate Z



In [49]:
#For each cube: folder,name, resolution cutoff, max/min amp (2 micron), min/max sigma, min/max mean, dimensions, max/min incidence, max/min emission, max/min phase, max/min lat/lon, 


np.max(nav[11])

90668.625

In [81]:
os.listdir('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

['2005359T100445_2005360T101919']

In [161]:
analyze_cube(data,nav,64,64)

({'max_real_amplitude': 0.14263956,
  'max_mean': 2.04040588452922,
  'max_sigma': 0.10240855324239037,
  'max_amplitude': 0.14444424521809296,
  'max_incidence': 99.923195,
  'max_emission': 89.56431,
  'max_phase': 29.267885,
  'max_lat': 65.747086,
  'max_lon': 359.9812},
 {'min_real_amplitude': 0.028945645,
  'min_mean': 2.031394333344775,
  'min_sigma': 0.04254957101394754,
  'min_amplitude': 0.02912475210617935,
  'min_incidence': 1.1092395,
  'min_emission': 0.7489744,
  'min_phase': 27.770727,
  'min_lat': -83.5026,
  'min_lon': 0.031011518})

In [174]:
def analyze_cube(cube,nav,x,y):
    """
    Given cube data (cube) and navigation information (nav) with dimensions x and y, returns:
    
    1) max/min 2 micron amplitude
    2) max/min sigma
    3) max/min mean 
    4) max/min incidence
    5) max/min emission
    6) max/min phase
    7) max/min lat
    8) max/min lon
    """
        
    maxes = {}
    mins = {}
    dimensions = (x,y)
    
    band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 
    
    for i in range(x):
        for j in range(y):
            #incidence and emission < 70
            if nav[6,i,j] < -90 and nav[2,i,j]<70 and nav[3,i,j]<70:
                continue
            else:
                try:
                    #Fit power law
                    p_fit, p_cov = sco.curve_fit(powerlaw, vims_wave[band_channels], cube[band_channels, i,j], p0 = (.005,-2,0), maxfev = 10000)
                    p = powerlaw(vims_wave, *p_fit)
                    
                    #Subtract power law
                    new_data = cube[:,i,j] - p

                    #Fit gaussian
                    fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],new_data[64:85], \
                                        p0=(2.08,.125,.1), maxfev=100000)

                    plaw[i,j] = p_fit[0]
                    plaw2[i,j] = p_fit[1]
                    plaw3[i,j] = p_fit[2]
                    
                except:
                    #If powerlaw fit fails, directly fit gaussian
                    try:
                        fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],cube[64:85,i,j], p0=(2.08,.125,.1), maxfev=1000000)
                    except:
                        break
                max_real_amp = np.max(cube[64:85,i,j])
                if maxes.get('max_real_amplitude',None) is None or maxes['max_real_amplitude'] < max_real_amp:
                    maxes['max_real_amplitude'] = max_real_amp
                if maxes.get('max_mean',None) is None or maxes['max_mean']< fit[0]:
                    maxes['max_mean']=fit[0]
                if maxes.get('max_sigma',None) is None or maxes['max_sigma'] <  fit[1]:
                    maxes['max_sigma'] = fit[1]
                if maxes.get('max_amplitude',None) is None or maxes['max_amplitude'] < fit[2]:
                    maxes['max_amplitude'] = fit[2]
                    
                if maxes.get('max_incidence',None) is None or maxes['max_incidence'] < nav[3,i,j]:
                    maxes['max_incidence'] = nav[3,i,j]
                if maxes.get('max_emission',None) is None or maxes['max_emission'] < nav[2,i,j]:
                    maxes['max_emission'] = nav[2,i,j]
                if maxes.get('max_phase',None) is None or maxes['max_phase'] < nav[1,i,j]:
                    maxes['max_phase'] = nav[1,i,j]
                if maxes.get('max_lat',None) is None or maxes['max_lat'] < nav[6,i,j]:
                    maxes['max_lat'] = nav[6,i,j]
                if maxes.get('max_lon',None) is None or maxes['max_lon'] < nav[7,i,j]:
                    maxes['max_lon'] = nav[7,i,j]
                    
                    
                min_real_amp = max_real_amp
                if mins.get('min_real_amplitude',None) is None or mins['min_real_amplitude'] > min_real_amp:
                    mins['min_real_amplitude'] = min_real_amp
                if mins.get('min_mean',None) is None or mins['min_mean']> fit[0]:
                    mins['min_mean']=fit[0]
                if mins.get('min_sigma',None) is None or mins['min_sigma'] >  fit[1]:
                    mins['min_sigma'] = fit[1]
                if mins.get('min_amplitude',None) is None or mins['min_amplitude'] > fit[2]:
                    mins['min_amplitude'] = fit[2]
                    
                if mins.get('min_incidence',None) is None or mins['min_incidence'] > nav[3,i,j]:
                    mins['min_incidence'] = nav[3,i,j]
                if mins.get('min_emission',None) is None or mins['min_emission'] > nav[2,i,j]:
                    mins['min_emission'] = nav[2,i,j]
                if mins.get('min_phase',None) is None or mins['min_phase'] > nav[1,i,j]:
                    mins['min_phase'] = nav[1,i,j]
                if mins.get('min_lat',None) is None or mins['min_lat'] > nav[6,i,j]:
                    mins['min_lat'] = nav[6,i,j]
                if mins.get('min_lon',None) is None or mins['min_lon'] > nav[7,i,j]:
                    mins['min_lon'] = nav[7,i,j]
                    
            
    
    return (maxes,mins)

In [178]:
def create_cube_parser(master_directory):
    '''
    Parameters:
    
    master_directory (str): Path to folder with all viewing geometries.
    
    Creates text file with information about each cube (assuming max resolution < 100km): 
    
    1) folder 
    2) name 
    3) max/min 2 micron amplitude
    4) max/min sigma
    5) max/min mean
    6) dimensions
    7) max/min incidence
    8) max/min emission
    9) max/min phase
    10) max/min lat
    11) max/min lon
    
    '''
    
    #file = open('cube_data.txt','w')
    skipped=set()
    viewing_geometries = os.listdir(master_directory)
    #print(viewing_geometries)
    for folder in viewing_geometries:
        file = open('cube_data.txt','a')
        print(f'Folder: {folder}')
        #file.write(f'{folder}\n')
        if folder == '.DS_Store':
            continue
        files = os.listdir(f'{master_directory}/{folder}')
        #print(files)
        for cube in files:
            
            #print(cube)
            if cube[0:2] != 'CM' or 'nav' in cube or 'BKGD' in cube:
                continue
                
            nav = pysis.CubeFile(f'{master_directory}/{folder}/{cube[:-4]}_nav.cub').data
            
            if np.max(nav[8])>100000:
                skipped.add((cube,'resolution'))
                continue
            
            print(f'Working on: {cube}')
            
            
                  
            name = cube
            
            data = pysis.CubeFile(f'{master_directory}/{folder}/{cube}').data
            
            lbl = pysis.CubeFile(f'{master_directory}/{folder}/{cube}').label
            
            desc = {'view':folder,'name':cube,'dimensions':(lbl['IsisCube']['Instrument']['SwathLength'],
                                    lbl['IsisCube']['Instrument']['SwathWidth'])}
            
            maxes,mins = analyze_cube(data,nav,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])
            
            #if maxes is empty, no meaningful data found
            
            if len(maxes)==0:
                skipped.add((cube,'lacking data'))
                continue
            
            
            desc.update(maxes)
            desc.update(mins)
                  
            file.write(f'{desc} \n')
        file.close()    
    #file.close()
    return skipped

In [177]:
create_cube_parser('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

Folder: 2009189T194600_2009189T235739
Working on: CM_1625786407_1_ir.cub
Working on: CM_1625776676_1_ir.cub
Working on: CM_1625777485_1_ir.cub
Working on: CM_1625777418_1_ir.cub
Working on: CM_1625777687_1_ir.cub
Working on: CM_1625780003_1_ir.cub
Working on: CM_1625777215_1_ir.cub
Working on: CM_1625777620_1_ir.cub
Working on: CM_1625778429_1_ir.cub
Working on: CM_1625778564_12_ir.cub
Working on: CM_1625788353_1_ir.cub
Working on: CM_1625776406_1_ir.cub
Working on: CM_1625785908_1_ir.cub
Working on: CM_1625783874_1_ir.cub
Working on: CM_1625778092_1_ir.cub
Working on: CM_1625778227_1_ir.cub
Working on: CM_1625776946_1_ir.cub
Working on: CM_1625777553_1_ir.cub
Working on: CM_1625785412_1_ir.cub
Working on: CM_1625777081_1_ir.cub
Working on: CM_1625782922_1_ir.cub
Working on: CM_1625776541_1_ir.cub
Working on: CM_1625776811_1_ir.cub
Working on: CM_1625776069_1_ir.cub
Working on: CM_1625780494_1_ir.cub
Working on: CM_1625784937_1_ir.cub
Working on: CM_1625777957_1_ir.cub
Working on: CM_1

{('CM_1514277149_1_ir.cub', 'lacking data'),
 ('CM_1514277541_1_ir.cub', 'resolution'),
 ('CM_1514280779_1_ir.cub', 'lacking data'),
 ('CM_1841286987_1_ir.cub', 'resolution'),
 ('CM_1841288670_1_ir.cub', 'resolution'),
 ('CM_1841288944_9_ir.cub', 'resolution'),
 ('CM_1841290630_1_ir.cub', 'resolution'),
 ('CM_1841290892_1_ir.cub', 'resolution'),
 ('CM_1841292710_1_ir.cub', 'resolution'),
 ('CM_1841292972_1_ir.cub', 'resolution'),
 ('CM_1841295610_5_ir.cub', 'resolution'),
 ('CM_1841346547_1_ir.cub', 'resolution'),
 ('CM_1841346809_1_ir.cub', 'resolution'),
 ('CM_1841351414_1_ir.cub', 'resolution'),
 ('CM_1841351567_1_ir.cub', 'resolution'),
 ('CM_1841355994_1_ir.cub', 'resolution'),
 ('CM_1841356068_1_ir.cub', 'resolution'),
 ('CM_1841367934_1_ir.cub', 'resolution'),
 ('CM_1841368069_1_ir.cub', 'resolution'),
 ('CM_1841376779_1_ir.cub', 'resolution'),
 ('CM_1841376895_1_ir.cub', 'resolution'),
 ('CM_1841387557_1_ir.cub', 'resolution'),
 ('CM_1841387657_8_ir.cub', 'resolution')}

In [171]:
data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841281547_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841281547_1_ir.cub').label
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/2016127T223449_2016130T200342/CM_1841281547_1_ir_nav.cub').data


analyze_cube(data,nav,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])

/home/alanyu/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


({'max_real_amplitude': 0.37406763,
  'max_mean': 44.480570945126644,
  'max_sigma': 19.042337266553897,
  'max_amplitude': 2.1609650266099266e+49,
  'max_incidence': 123.39848,
  'max_emission': 89.74572,
  'max_phase': 120.99549,
  'max_lat': 88.63443,
  'max_lon': 359.8052},
 {'min_real_amplitude': 0.00032065713,
  'min_mean': -41.72438779902643,
  'min_sigma': -3.0424541052519247,
  'min_amplitude': -0.01103393520521871,
  'min_incidence': 32.634655,
  'min_emission': 0.92859846,
  'min_phase': 120.14005,
  'min_lat': 16.058416,
  'min_lon': 0.41456932})

In [13]:
def read_database(file_name):
    data = {}
    
    file = open(file_name,'r')
    while True:
        try:
            temp = ast.literal_eval(file.readline())
            name = temp['name']
            del(temp['name'])
            data.update({name:temp})
        except Exception as e:
            print(e)
            break
            
    file.close()
    
    return data

In [15]:
read_database('cube_data.txt')

unexpected EOF while parsing (<unknown>, line 0)


523

In [12]:
dat = {}
f = open('cube_data.txt','r')
temp = ast.literal_eval(f.readline())
name = temp['name']
del(temp['name'])
dat.update({name:temp})
dat

{'CM_1838530943_1_ir.cub': {'view': '2016096T054156_2016097T040514',
  'dimensions': (64, 46),
  'max_real_amplitude': 0.4440777,
  'max_mean': 30.7460228728227,
  'max_sigma': 5.604866381344292,
  'max_amplitude': 1.9206460834707575e+39,
  'max_incidence': 167.5496,
  'max_emission': 89.46845,
  'max_phase': 128.90904,
  'max_lat': 89.19383,
  'max_lon': 354.01587,
  'min_real_amplitude': 6.1433246e-05,
  'min_mean': -52.59100393397263,
  'min_sigma': -2.5958860532514874,
  'min_amplitude': -9.375792693159017e+33,
  'min_incidence': 44.918026,
  'min_emission': 0.15046662,
  'min_phase': 127.5899,
  'min_lat': -32.07777,
  'min_lon': 8.737222}}